In [59]:
from transformers import pipeline 
import torch
import pandas as pd

PATH = "C:\Emotion Classification\data\\full_dataset\goemotions_1.csv"

df = pd.read_csv(PATH)
df.head(1)

<>:5: SyntaxWarning: invalid escape sequence '\E'
<>:5: SyntaxWarning: invalid escape sequence '\E'
C:\Users\illya\AppData\Local\Temp\ipykernel_20436\2040331901.py:5: SyntaxWarning: invalid escape sequence '\E'
  PATH = "C:\Emotion Classification\data\\full_dataset\goemotions_1.csv"


,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1.548381e+09,1,False,0,...,0,0,0,0,0,0,0,1,0,0


In [60]:
df = df.drop(columns=['id', 'author', 'subreddit', 'link_id', 'parent_id',
       'created_utc', 'rater_id'])

In [61]:
generator = pipeline("text-classification")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [62]:
df.text[0]

'That game hurt.'

In [63]:
generator(df.text[0])

[{'label': 'NEGATIVE', 'score': 0.9996750354766846}]

In [64]:
result = generator(df.text[0])
result[0]['label']

'NEGATIVE'

In [65]:
classifier = pipeline(
    "sentiment-analysis", 
    device=0 if torch.cuda.is_available() else -1
)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [66]:
batch_size = 64
texts = df['text'].fillna('').astype(str).tolist()
all_results = []

In [67]:
from tqdm import tqdm

In [68]:
for i in tqdm(range(0, len(texts), batch_size)):
    batch = texts[i:i+batch_size]
    results = classifier(batch)
    all_results.extend(results)

pos_counter = sum(1 for r in all_results if r['label'] == 'POSITIVE')
neg_counter = sum(1 for r in all_results if r['label'] == 'NEGATIVE')

print(f"Позитивних: {pos_counter}, Негативних: {neg_counter}")

100%|██████████| 1094/1094 [35:37<00:00,  1.95s/it]

Позитивних: 27721, Негативних: 42279


In [69]:
df.columns 

Index(['text', 'example_very_unclear', 'admiration', 'amusement', 'anger',
       'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire',
       'disappointment', 'disapproval', 'disgust', 'embarrassment',
       'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love',
       'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse',
       'sadness', 'surprise', 'neutral'],
      dtype='object')

In [70]:
sentiment_analyzer = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")

Device set to use cpu


In [71]:
sentiment_analyzer(df.text[3])

[{'label': 'LABEL_2', 'score': 0.971968948841095}]

In [72]:
positive_emotions = [
    "admiration", "amusement", "approval", "caring", "curiosity", 
    "desire", "excitement", "gratitude", "joy", "love", 
    "optimism", "pride", "relief"
]

negative_emotions = [
    "anger", "annoyance", "confusion", "disappointment", "disapproval", 
    "disgust", "embarrassment", "fear", "grief", "nervousness", 
    "remorse", "sadness"
]

neutral_emotions = [
    "example_very_unclear", "realization", "surprise", "neutral"
]

In [73]:
df = df[~(df[neutral_emotions] == 1).any(axis=1)]
df = df.reset_index(drop=True)

In [74]:
df.shape

(45874, 30)

In [75]:
df.columns

Index(['text', 'example_very_unclear', 'admiration', 'amusement', 'anger',
       'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire',
       'disappointment', 'disapproval', 'disgust', 'embarrassment',
       'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love',
       'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse',
       'sadness', 'surprise', 'neutral'],
      dtype='object')

In [76]:
df = df.drop(columns=neutral_emotions, axis=1)

In [77]:
df.shape, df.columns

((45874, 26),
 Index(['text', 'admiration', 'amusement', 'anger', 'annoyance', 'approval',
        'caring', 'confusion', 'curiosity', 'desire', 'disappointment',
        'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear',
        'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride',
        'relief', 'remorse', 'sadness'],
       dtype='object'))

In [ ]:
df_positive = df[(df[positive_emotions] == 1).any(axis=1) & ~(df[negative_emotions] == 1).any(axis=1)]
df_positive = df_positive.drop(columns=negative_emotions, axis=1)
df_negative = df[(df[negative_emotions] == 1).any(axis=1) & ~(df[positive_emotions] == 1).any(axis=1)]
df_negative = df_negative.drop(columns=positive_emotions, axis=1)

(43332, 13)

In [82]:
df_positive.shape[0] + df_negative.shape[0], df_negative.shape[1] + df_positive.shape[1]

(43332, 27)

In [79]:
df_positive.to_csv("C:\Emotion Classification\\treeModelsData\positive_emotions.csv", sep=',' ,index=False)
df_negative.to_csv("C:\Emotion Classification\\treeModelsData\\negative_emotions.csv", sep=',' ,index=False)

<>:1: SyntaxWarning: invalid escape sequence '\E'
<>:2: SyntaxWarning: invalid escape sequence '\E'
<>:1: SyntaxWarning: invalid escape sequence '\E'
<>:2: SyntaxWarning: invalid escape sequence '\E'
C:\Users\illya\AppData\Local\Temp\ipykernel_20436\2756853363.py:1: SyntaxWarning: invalid escape sequence '\E'
  df_positive.to_csv("C:\Emotion Classification\\treeModelsData\positive_emotions.csv", sep=',' ,index=False)
C:\Users\illya\AppData\Local\Temp\ipykernel_20436\2756853363.py:2: SyntaxWarning: invalid escape sequence '\E'
  df_negative.to_csv("C:\Emotion Classification\\treeModelsData\\negative_emotions.csv", sep=',' ,index=False)


In [80]:
df_positive.columns

Index(['text', 'admiration', 'amusement', 'approval', 'caring', 'curiosity',
       'desire', 'excitement', 'gratitude', 'joy', 'love', 'optimism', 'pride',
       'relief'],
      dtype='object')

In [81]:
df_negative.columns

Index(['text', 'anger', 'annoyance', 'confusion', 'disappointment',
       'disapproval', 'disgust', 'embarrassment', 'fear', 'grief',
       'nervousness', 'remorse', 'sadness'],
      dtype='object')